In [11]:
import pandas as pd
import geopandas as gpd

In [3]:
trips = pd.read_csv(r"T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\trips_org_dst.csv")

In [4]:
trips.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,Distance,Minutes,OriginDestination,Weekday,Month,Season,SeasonOrder,WeekdayOrder,Year
1531533,14824759,19600,2218937,NaN,44.060020,-123.057868,2022-12-31,21:55,3.37,22.716667,Destination,Saturday,December,Winter,4,6,2022
1531534,14824763,19717,919298,PeaceHealth University District,44.047618,-123.081802,2022-12-31,22:05,1.25,8.416667,Destination,Saturday,December,Winter,4,6,2022
1531535,14824768,19828,2300240,17th & Hilyard,44.041130,-123.081808,2022-12-31,22:19,0.39,3.266667,Destination,Saturday,December,Winter,4,6,2022
1531536,14824771,19632,2267182,Matthew Knight Arena,44.045390,-123.065177,2022-12-31,22:31,1.66,12.466667,Destination,Saturday,December,Winter,4,6,2022
1531537,14824790,19658,2208273,Franklin Blvd & Walnut North,44.045292,-123.061408,2022-12-31,23:28,1.15,6.666667,Destination,Saturday,December,Winter,4,6,2022


In [5]:
trips.Year.unique()

array([2019, 2020, 2021, 2022, 2023, 2018], dtype=int64)

In [6]:
excludedIDs = [717565, 742339, 764038, 819845, 1228447, 1354709, 1897910, 2184703, 2207685]

In [7]:
excludedyears = [2018, 2023]

In [8]:
trips = trips[(~trips.UserID.isin(excludedIDs))&(~trips.Year.isin(excludedyears))]

In [9]:
trips.Location = trips.Location.str.replace(', ', ' @ ', regex=True)

In [10]:
trips.Location = trips.Location.str.replace('Monroe St & Blair Blvd ', 'Monroe St & Blair Blvd', regex=True)

In [12]:
stations = gpd.read_file(r"T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeMap\data\mapdata\bike_share_stations.shp")

In [13]:
stations.head()

,station id,name,geometry
0,hub_3828,Eugene Family YMCA,POINT (-13701591.712 5471253.204)
1,hub_3829,Monroe St & Blair Blvd,POINT (-13704001.270 5473411.225)
2,hub_3830,10th & Olive SE,POINT (-13702770.365 5472975.425)
3,hub_3831,16th & Willamette,POINT (-13702642.423 5471973.384)
4,hub_3832,13th & Pearl @ Pearl side,POINT (-13702278.750 5472449.473)


In [14]:
"Monroe St & Blair Blvd" in stations.name.values

True

In [15]:
tripLocations = [x for x in trips.Location.unique() if x != 'nan']

In [16]:
tripLocSelected = []
Loc2Review = []

In [17]:
# check if the trip location names are matched with the station names
for loc in tripLocations:
    if loc in stations.name.values:
        tripLocSelected.append(loc)
    else:
        print(loc)
        Loc2Review.append(loc)

nan
17th & Pearl
16th & Hilyard @ West side
Millrace Garage
Event Rack 1 (Virtual Hub)
Outdoor Program Barn (Virtual Hub)
16th & Hilyard @ East side
10th & Olive @ South
10th & Olive @ North
Court Cafe (virtual hub)
Carson Hall (virtual hub)
Franklin Blvd & Walnut North
Autzen Stadium (Virtual Hub)
Papé Field (Virtual Hub)
17th & Pearl @ NE corner
16th & Hilyard
Frohnmayer Footbridge
Monroe & Blair
Lane County Farmer's Market (Virtual Hub)
8th & Park
NEST
Heartfelt House (Virtual Hub)
17th & Pearl @ SW corner
Unthank Hall (Virtual Hub)
12th & Patterson (Virtual Hub)
13th & Oak (Virtual Hub)
13th & Olive (Virtual Hub)
12th & Patterson Alley (Virtual Hub)
Broadway & Hilyard (Virtual Hub)
17th & Ferry Alley


In [18]:
trips_selected = trips[trips.Location.isin(tripLocSelected)]

In [19]:
originCounts = trips_selected[trips_selected.OriginDestination == 'Origin'][['Location', 'RouteID', 'UserID']].groupby('Location').agg({"RouteID": lambda x: x.nunique(), "UserID": lambda x: x.nunique()})

In [20]:
originCounts.head()

,RouteID,UserID
Location,,
10th & Olive NE,315,146
10th & Olive SE,6313,1393
10th & Pearl,2703,1019
12th & Willamette,19725,2723
13th & Agate (Virtual Hub),2853,1042


In [21]:
len(trips_selected.UserID.unique())

29468

In [22]:
trips_selected.shape[0]

767244

In [23]:
data = {'name':originCounts.index.values,
        'ntrips':originCounts.RouteID.values,
        'nbikers':originCounts.UserID.values}

In [24]:
df = pd.DataFrame(data)

In [25]:
origin_df = stations.merge(df, how='inner', on='name')

In [26]:
origin_df.head()

,station id,name,geometry,ntrips,nbikers
0,hub_3828,Eugene Family YMCA,POINT (-13701591.712 5471253.204),12255,1623
1,hub_3829,Monroe St & Blair Blvd,POINT (-13704001.270 5473411.225),2145,632
2,hub_3830,10th & Olive SE,POINT (-13702770.365 5472975.425),6313,1393
3,hub_3831,16th & Willamette,POINT (-13702642.423 5471973.384),4940,1096
4,hub_3832,13th & Pearl @ Pearl side,POINT (-13702278.750 5472449.473),5824,1840


In [27]:
origin_df.to_file(r'T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\OriginCounts.shp')

In [28]:
destinationCounts = trips_selected[trips_selected.OriginDestination == 'Destination'][['Location', 'RouteID', 'UserID']].groupby('Location').agg({"RouteID": lambda x: x.nunique(), "UserID": lambda x: x.nunique()})

In [29]:
data = {'name':destinationCounts.index.values,
        'ntrips':destinationCounts.RouteID.values,
        'nbikers':destinationCounts.UserID.values}
df = pd.DataFrame(data)
destination_df = stations.merge(df, how='inner', on='name')
destination_df.to_file(r'T:\MPO\Bike&Ped\BikeCounting\StoryMap\BikeShare\Data\DestinationCounts.shp')